<a href="https://colab.research.google.com/github/papagorgio23/Python101/blob/master/Email_Notification_Template_Update_to_GBQ_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pydata_google_auth 

In [0]:
import pandas as pd
import pandas_gbq
import pydata_google_auth
import smtplib
import sys


#authenticate to GBQ
SCOPES = ['https://www.googleapis.com/auth/cloud-platform',
          'https://www.googleapis.com/auth/drive']

credentials = pydata_google_auth.get_user_credentials(SCOPES, auth_local_webserver=False)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=BCwDJVEBSYfyZBPraigjiZq6DnxH6k&prompt=consent&access_type=offline&code_challenge=UIYMWgm1Ei1Ngip_U1EXnG8up_GwOsBNeVr57xC01p0&code_challenge_method=S256
Enter the authorization code: 4/qQFowvIgRlBr6cDK48Lwto9IrUc_OpFLqRGthvGLIgZpVH5E6X57GQ8


In [0]:
#pull data from GBQ and convert to df

sql = '''
with data as (
select 
lead_partner
,min(date(qb_created_datetime)) as created_date
from `fdr-dp-stg.standardized_data.lead_details`
where crm_source = 'suip'
and test_leads_flag_suip = 0
group by lead_partner
)

select *
from data
where created_date >='2019-08-01'
order by created_date
'''

projectId = 'ffn-dw-bigquery-prd'

#run query and save to a variable called df (use pandas_gbq.read_gbq)
#params: query, project_id, credentials, dialect
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_gbq.html
df = pandas_gbq.read_gbq(query = sql,
          project_id = projectId,
          credentials=credentials, 
          dialect='standard')

In [0]:
#exit script if df is empty
if df.empty:
  sys.exit()

In [0]:
#create a variable called html for message body of email
#add a greeting (<p>) and start table tags to hold data. <tr> with two <th>
#https://www.w3schools.com/html/
html = '''
<p>Hello,</p>
<p>There are new lead partners going to SUIP.</p>

<table>
  <tr>
    <th>Lead Partner</th>
    <th>Created Date</th>
  </tr>
'''



In [0]:
#loop through dataframe to add lead partner data to HTML table
#for each row add one <tr> and two <td> for lead_partner and MinCreatedDate
#use <date>.strftime('%m/%d/%Y') for date formatting
for row in df.itertuples():
  html += '<tr>'
  html += '<td>' + row.lead_partner + '</td>'
  html += '<td>' + row.created_date.strftime('%m%d%Y') + '</td>'
  html += '</tr>'

html += '</table>'


#add disclosure to html with page breaks
html += '<br><br><br><br> This is an automated message. Email Jason@AISportsFirm.com with any questions.'

print(html)


<p>Hello,</p>
<p>There are new lead partners going to SUIP.</p>

<table>
  <tr>
    <th>Lead Partner</th>
    <th>Created Date</th>
  </tr>
<tr><td>wfs_yhp_ios</td><td>08012019</td></tr><tr><td>33</td><td>08062019</td></tr><tr><td>185</td><td>08082019</td></tr><tr><td>spop_rmk</td><td>08082019</td></tr><tr><td>113</td><td>08082019</td></tr><tr><td>fdrreviews.com</td><td>08092019</td></tr><tr><td>nextdoor</td><td>08122019</td></tr><tr><td>4</td><td>08152019</td></tr><tr><td>5</td><td>08192019</td></tr><tr><td>wfs_yhp_and</td><td>08202019</td></tr><tr><td>DirectMailWF</td><td>08202019</td></tr><tr><td>outbrain</td><td>08202019</td></tr><tr><td>222</td><td>08212019</td></tr></table><br><br><br><br> This is an automated message. Email Jason@AISportsFirm.com with any questions.


<p>Hello,</p>
<p>There are new lead partners going to SUIP.</p>

<table>
  <tr>
    <th>Lead Partner</th>
    <th>Created Date</th>
  </tr>
<tr><td>wfs_yhp_ios</td><td>08012019</td></tr><tr><td>33</td><td>08062019</td></tr><tr><td>185</td><td>08082019</td></tr><tr><td>spop_rmk</td><td>08082019</td></tr><tr><td>113</td><td>08082019</td></tr><tr><td>fdrreviews.com</td><td>08092019</td></tr><tr><td>nextdoor</td><td>08122019</td></tr><tr><td>4</td><td>08152019</td></tr><tr><td>5</td><td>08192019</td></tr><tr><td>wfs_yhp_and</td><td>08202019</td></tr><tr><td>DirectMailWF</td><td>08202019</td></tr><tr><td>outbrain</td><td>08202019</td></tr><tr><td>222</td><td>08212019</td></tr></table><br><br><br><br> This is an automated message. Email Jason@AISportsFirm.com with any questions.

In [0]:
#build and send email
#to build email message: https://docs.python.org/2/library/email-examples.html
#to send the email:      https://docs.python.org/3/library/smtplib.html

#import MIMEMultipart and MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


#define email addresses to use (fromEmail and toEmail)
fromEmail = 'jalee@freedomplus.com'
toEmail = 'jason@AISportsFirm.com'



#build message then ad Subject, From, and To
message = MIMEMultipart('alternative')
message['Subject'] = 'New Lead Partner'
message['From'] = fromEmail
message['To'] = toEmail


#create MIMEText object and attach html
messageBody = MIMEText(html, 'html')
message.attach(messageBody)

#connect to Outlook email server, login and send
mailserver = smtplib.SMTP('smtp.office365.com',587)
mailserver.ehlo()
mailserver.starttls()
mailserver.login('JaLee@freedomdebtrelief.com', password)
mailserver.sendmail(fromEmail, toEmail, message.as_string())
mailserver.quit()

(221, b'2.0.0 Service closing transmission channel')

In [0]:
from datetime import datetime
from google.colab import files 

#download df to a csv file and save with today's date
#create a variable called today (use datetime.now())
today = datetime.now().strftime('%m.%d.%Y')

#convert df to csv and download from google colab with files.download
filename = 'Lead Partner_' + today
df.to_csv(filename)
files.download(filename)


In [0]:
#fix column data types to match GBQ data type
#lead_partner to string and created_date to date
df['lead_partner'] = df['lead_partner'].astype('str')

In [0]:
#add a column row_loaded_date_time with current timestamp
df['row_loaded_date_time'] = datetime.now()
df

,lead_partner,created_date,row_loaded_date_time
0,wfs_yhp_ios,2019-08-01,2019-08-23 21:40:41.774247
1,33,2019-08-06,2019-08-23 21:40:41.774247
2,185,2019-08-08,2019-08-23 21:40:41.774247
3,spop_rmk,2019-08-08,2019-08-23 21:40:41.774247
4,113,2019-08-08,2019-08-23 21:40:41.774247
5,fdrreviews.com,2019-08-09,2019-08-23 21:40:41.774247
6,nextdoor,2019-08-12,2019-08-23 21:40:41.774247
7,4,2019-08-15,2019-08-23 21:40:41.774247
8,5,2019-08-19,2019-08-23 21:40:41.774247
9,wfs_yhp_and,2019-08-20,2019-08-23 21:40:41.774247


In [0]:
#set destination table to insert data into
destinationtable = 'NewTable_location'

#send data to GBQ (pandas_gqb.to_gbq)
#https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
#params are: dataframe, destination_table, project_id, if_exists (append), and table_schema (list of dicts, name and type)
pandas_gbq.to_gbq(dataframe = df,
                  destination_table = destinationtable,
                  project_id = project_id,
                  if_exists = 'append',
                  table_schema = [{'name':'lead_partner',
                                   'type':'STRING'},
                                  {'name':'created_date',
                                   'type':'date'},
                                  {'name':'row_loaded_date_time',
                                   'type':'TIMESTAMP'}])